W niniejszym projekcie wykonano analizę zbioru [South German Credit Data Set](http://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29) pochodzącego z serwisu [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets.php). Dane w tym zbiorze dotyczą problemu udzielania kredytów, a dokładniej pozwalają określić ryzyko kredytowe na podstawie danych o kredytobiorcach, takich jak: wiek, zatrudnienie czy sytuacja mieszkaniowa. Dane pochodzą z lat 1973-1975.

# Prezentacja zbioru

Działania rozpoczęto od zaimportowania bibliotek:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_daq as daq
import dash_html_components as html
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

sns.set_theme(style='darkgrid')
JupyterDash.infer_jupyter_proxy_config()

Następnie, wczytano zbiór danych, w którym separatorem jest spacja:

In [ ]:
df = pd.read_table('data/SouthGermanCredit.asc', sep=' ')

Jako, że w zbiorze zostały wykorzystane niemieckie nazwy kolumn, postanowiono zamienić je na angielskie odpowiedniki, które były zawarte w dokumentacji zbioru:

In [ ]:
df.rename(columns={
    'laufkont': 'status',
    'laufzeit': 'duration',
    'moral': 'credit_history',
    'verw': 'purpose',
    'hoehe': 'amount',
    'sparkont': 'savings',
    'beszeit': 'employment_duration',
    'rate': 'installment_rate',
    'famges': 'personal_status_sex',
    'buerge': 'other_debtors',
    'wohnzeit': 'present_residence',
    'verm': 'property',
    'alter': 'age',
    'weitkred': 'other_installment_plans',
    'wohn': 'housing',
    'bishkred': 'number_credits',
    'beruf': 'job',
    'pers': 'people_liable',
    'telef': 'telephone',
    'gastarb': 'foreign_worker',
    'kredit': 'credit_risk'
}, inplace=True)

Podgląd pierwszych pięciu rekordów:

In [ ]:
df.head()

Zbiór składa się z 1000 wierszy i 21 kolumn:

In [ ]:
df.shape

Statystyka podsumowująca zbiór:

In [ ]:
df.describe()

W zbiorze nie występują brakujące wartości:

In [ ]:
df.isnull().any()

# Analiza wartości na poszczególnych kolumnach

Jako, że ten zbiór składa się w większości z danych jakościowych, wartości te zostały zapisane w zbiorze (przez autora zbioru) za pomocą odpowiadających im kluczy, w postaci kolejnych liczb naturalnych, a ich znaczenie zostało opisane osobno dla każdej kolumny poniżej.

Atrybutem decyzyjnym w tym zbiorze jest `credit_risk`, który przyjmuje jedną z dwóch wartości: `0` albo `1`. Pierwsza oznacza, że klient przestrzegał ustaleń i należycie spłacał zadłużenie, a druga, że nie przestrzegał ustaleń i występowały problemy ze spłatą zadłużenia.

## 1. Kolumna `status`

Na kolumnie `status` przechowywane są informacje na temat rachunków rozliczeniowych kredytobiorców. Znaczenie poszczególnych wartości:

* **1** - brak konta rozliczeniowego,
* **2** - mniej niż 0 DM\* na koncie rozliczeniowym,
* **3** - 0 DM lub więcej, ale mniej niż 200 DM na koncie rozliczeniowym,
* **4** - więcej niż 200 DM na koncie rozliczeniowym albo pensja od co najmniej roku.

\* DM - Deutsche Mark - marka niemiecka (waluta)

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['status'].isin([1, 2, 3, 4])).any()

Po analizie powiązań między wartościami na kolumnie `status`, a ryzykiem kredytowym można wysnuć wniosek, że im więcej pieniędzy na koncie rozliczeniowym posiadał klient, tym miał większe problemy ze spłatą zadłużenia (wyjątkiem jest tutaj sytuacja kiedy klient posiadał 0 DM lub więcej, ale mniej niż 200 DM na koncie rozliczeniowym - wynika to raczej z mniejszej liczby rekordów o tej wartości).

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='status', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)

## 2. Kolumna `duration`

Na kolumnie `duration` przechowywane są informacje o długości spłaty zadłużenia kredytobiorców, wyrażone w liczbie miesięcy. Wartości należą do przedziału `[4, 72]`.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do wyżej wymienionego przedziału:

In [ ]:
np.logical_not(df['duration'].between(4, 73)).any()

Poniżej umieszczono wykres przedstawiający liczbę zaciągniętych kredytów względem długości spłaty zadłużenia (wartości wyrażone w liczbie miesięcy). Najpopularniejszym wyborem okazało się zobowiązanie na czas do 12 miesięcy. Można także zauważyć, że im dłuższy czas spłaty zobowiązania tym mniejsze zainteresowanie kredytobiorców.

In [ ]:
data = df['duration'].value_counts(bins=pd.interval_range(start=4, freq=8.5, end=72), sort=False)

indexes = data.index
values = data.values
x_positions = np.arange(len(indexes))

plt.figure(figsize=(7, 5))
plt.title('Popularność długości czasu spłaty kredytów')
plt.xlabel('Czas spłaty kredytu (w miesiącach)')
plt.ylabel('Liczba kredytów')
plt.bar(x_positions, values)
plt.xticks(x_positions, indexes, rotation=90)
plt.show()

## 3. Kolumna `credit_history`

Na kolumnie `credit_history` zawarte są informacje o historiach kredytowych kredytobiorców. Znaczenie poszczególnych wartości:

* **0** - opóźnienia w spłacie w przeszłości,
* **1** - konto krytyczne / kredyt w innym miejscu,
* **2** - brak zaciągniętych kredytów,
* **3** - dotychczasowe kredyty spłacane należycie,
* **4** - wszystkie kredyty w tym banku spłacone należycie.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['credit_history'].isin([0, 1, 2, 3, 4])).any()

Analizując powiązania pomiędzy danymi na kolumnie `credit_history`, a ryzykiem kredytowym można zauważyć, że największe problemy ze spłatą kredytu mieli klienci, którzy wcześniej nie zaciągali kredytów. Zaskakująca może być także informacja, że często występowały też problemy z klientami, którzy do tej pory spłacali swoje zobowiązania bez zarzutów.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='credit_history', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 4. Kolumna `purpose`

Na kolumnie `purpose` zawarte są informacje o celach, na które zaciągano kredyt. Znaczenie poszczególnych wartości:

* **0** - inne,
* **1** - samochód z salonu,
* **2** - samochód używany,
* **3** - meble/wyposażenie,
* **4** - radio/telewizor,
* **5** - sprzęt gospodarstwa domowego,
* **6** - remont,
* **7** - edukacja,
* **8** - wakacje,
* **9** - przekwalifikowanie,
* **10** - biznes.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['purpose'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])).any()

Po analizie danych na kolumnie `purpose` można zauważyć, że trzema najczęstszymi powodami zaciągnięcia kredytu były: kupno mebli/wyposażenia, kupno samochodu używanego oraz inne. Trzema najrzadszymi powodami zaciągnięcia kredytu były: wakacje, kupno radia/telewizora oraz biznes.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='purpose')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 5. Kolumna `amount`

Na kolumnie `amount` zawarte są informacje o wysokości kredytów wyrażone w markach niemieckich (DM).

Poniżej umieszczono wykres przedstawiający popularność kwot kredytów. Można zauważyć, że najwięcej kredytów udzielono na kwoty do ok. 2500 DM, popularne były także kredyty powyżej ok. 2500 DM do ok. 5000 DM. Wyższe kwoty były zdecydowanie mniej popularne.

In [ ]:
data = df['amount'].value_counts(bins=pd.interval_range(start=250, freq=2271.75, end=18424), sort=False)

indexes = data.index
values = data.values
y_positions = np.arange(len(values))

plt.figure(figsize=(7, 5))
plt.title('Liczba udzielonych kredytów z podziałem na kwoty')
plt.xlabel('Liczba kredytów')
plt.ylabel('Kwota kredytu')
plt.barh(y_positions, values)
plt.gca().invert_yaxis()
plt.yticks(y_positions, indexes)
plt.show()

## 6. Kolumna `savings`

Na kolumnie `savings` zawarte są informacje o oszczędnościach kredytobiorców. Znaczenia poszczególnych wartości:

* **1** - brak danych / brak oszczędności
* **2** - poniżej 100 DM\*,
* **3** - od 100 DM do 500 DM,
* **4** - od 500 DM do 1000 DM,
* **5** - powyżej 1000 DM.


\* DM - Deutsche Mark - marka niemiecka (waluta)

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['savings'].isin([1, 2, 3, 4, 5])).any()

Po analize danych z kolumny `savings` można zauważyć, że największe problemy ze spłatą zadłużenia mieli klienci, którzy nie mieli żadnych oszczędności. Zaskakująca może być także informacja, że dość dużo, bo aż 151 klientów, którzy posiadali ponad 1000 DM oszczędności, mieli problemy ze spłatą zadłużenia.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='savings', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 7. Kolumna `employment_duration`

Na kolumnie `employment_duration` zawarte są informacje o długości zatrudnienia kredytobiorców u ówczesnego pracodawcy. Znaczenia poszczególnych wartości:

* **1** - bezrobotny,
* **2** - poniżej roku,
* **3** - od roku do 4 lat,
* **4** - od 4 do 7 lat,
* **5** - powyżej 7 lat.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['employment_duration'].isin([1, 2, 3, 4, 5])).any()

Po analizie danych na kolumnie `employment_duration` można zauważyć, że największe problemy ze spłatą zadłużenia mieli klienci, którzy pracowali u ówczesnego pracodawcy od roku do 4 lat. Można także zauważyć, że najmniejsze problemy występowały z klientami bezrobotnymi, ale wynika to raczej z małej liczby takich klientów.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='employment_duration', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 8. Kolumna `installment_rate`

Na kolumnie `installment_rate` zawarte są informacje o wysokości miesięcznych rat wyrażone jako procent od pensji kredytobiorców. Znaczenia poszczególnych wartości:

* **1** - >= 35%,
* **2** - 25% <= ... < 35%,
* **3** - 20% <= ... < 25%,
* **4** - < 20%.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['installment_rate'].isin([1, 2, 3, 4])).any()

Analiza danych na kolumnie `installment_rate` doprowadziła do powstania zaskakującego wniosku: największe problemy ze spłatą zadłużenia mieli klienci, u których miesięczna rata kredytu stanowiła mniej niż 20% pensji. U klientów, u których rata kredytu stanowi więcej niż 20% pensji występowały mniejsze problemy ze spłatą zadłużenia.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='installment_rate', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 9. Kolumna `personal_status_sex`

Na kolumnie `personal_status_sex` zawarte są informacje o płci i statusie matrymonialnym kredytobiorców. Znaczenie poszczególnych wartości:

* **1** - mężczyzna: rozwiedziony/w separacji,
* **2** - kobieta: nie samotna/mężczyzna: samotny,
* **3** - mężczyzna: żonaty/wdowiec,              
* **4** - kobieta: samotna.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['personal_status_sex'].isin([1, 2, 3, 4])).any()

Po analizie wartości na kolumnie `personal_status_sex` okazało się, że największe problemy ze spłatą zadłużenia mieli mężczyźni, którzy byli żonaci albo byli wdowcami. Często problemy występowały także z nie samotnymi kobietami.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='personal_status_sex', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

W przypadku tej kolumny postanowiono wyodrębnić dodatkowe informacje na temat płci kredytobiorców do osobnej kolumny (oznaczenia: 0 - mężczyzna, 1 - kobieta):

In [ ]:
df['sex'] = np.where(df['personal_status_sex'].isin([1, 3]), 0, 1)

Poniżej przedstawiono wykres kołowy przedstawiający udział płci kredytobiorców. Wśród kredytobiorców więcej było mężczyzn, bo prawie 60%. Kobiety stanowiły 40%.

In [ ]:
data = df['sex'].value_counts().rename(index={0: 'Mężczyżni', 1: 'Kobiety'})

indexes = data.index
values = data.values

plt.figure(figsize=(7, 5))
plt.title('Procentowy udział płci kredytobiorców')
plt.pie(values, labels=indexes, autopct='%1.2f%%')
plt.axis('equal')
plt.show()

## 10. Kolumna `other_debtors`

Na kolumnie `other_debtors` zawarte są informacje o powiązanych dłużnikach. Znaczenie poszczególnych wartości:

* **1** - brak,
* **2** - współwnioskodawca,
* **3** - żyrant.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['other_debtors'].isin([1, 2, 3])).any()

Można zauważyć, że większość kredytów była związana z tylko jedną osobą (bez współwnioskodawców czy żyrantów). Pozostałe przypadki występowały rzadko.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='other_debtors')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 11. Kolumna `present_residence`

Na kolumnie `present_residence` zawarte są informacje o czasie zamieszkania w ówczesnym miejscu zamieszkania kredytobiorców. Znaczenie poszczególnych wartości:

* **1** - mniej niż rok,
* **2** - od roku do 4 lat,
* **3** - od 4 do 7 lat,
* **4** - powyżej 7 lat.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['present_residence'].isin([1, 2, 3, 4])).any()

Analiza danych na kolumnie `present_residence` wykazała, że im dłużej klient mieszkał w danym miejscu, tym większe było prawdopodobieństwo problemów ze spłatą zadłużenia.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='present_residence', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 12. Kolumna `property`

Na kolumnie `property` zawarte są informacje o najdroższych posiadanych własnościach przez kredytobiorców. Znaczenie poszczególnych wartości:

* **1** - nie wiadomo / brak własności,
* **2** - samochód albo inne,
* **3** - mieszkanie ze spółdzielni mieszkaniowej, umowa oszczędnościowa / ubezpieczenie na życie,
* **4** - nieruchomość.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['property'].isin([1, 2, 3, 4])).any()

Analizując dane na kolumnie `property` można dojść do wniosku, że klienci, którzy posiadali nieruchomość mieli zazwyczaj mniejsze problemy ze spłatą zadłużenia.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='property', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 13. Kolumna `age`

Na kolumnie `age` zawarte są informacje o wieku kredytobiorców.

Poniżej przedstawiono histogram prezentujący informacje o liczbie kredytobiorców w danym wieku.

In [ ]:
plt.figure(figsize=(7, 5))
plt.title('Histogram przedstawiający wiek kredytobiorców')
plt.xlabel('Wiek')
plt.ylabel('Liczba kredytobiorców')
plt.hist(df['age'])
plt.show()

Można zauważyć, że najczęściej kredytobiorcami były osoby w wieku 25-35 lat. Popularność kredytów wśród osób starszych niż 35 lat spada, aż do najmniejszej wartości dla osób w wieku powyżej 70 lat.

## 14. Kolumna `other_installment_plans`

Na kolumnie `other_installment_plans` zawarte są informacje o innych zobowiązaniach kredytowych klientów. Znaczenie poszczególnych wartości:

* **1** - bank,
* **2** - sklep,
* **3** - brak.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['other_installment_plans'].isin([1, 2, 3])).any()

Po analizie danych na kolumnie `other_installment_plans` okazuje się, że posiadanie innych zobowiązań kredytowych często nie jest równoznaczne z problemami ze spłatą kredytu przez klientów. Problem ze spłatą kredytów znacznie częściej występował wśród klientów, którzy nie mieli innych zobowiązań.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='other_installment_plans', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 15. Kolumna `housing`

Na kolumnie `housing` zawarte są informacje o rodzaju zakwaterowania kredytobiorców. Znaczenie poszczególnych wartości:

* **1** - mieszkanie za darmo,
* **2** - mieszkanie wynajmowane,
* **3** - mieszkanie własnościowe.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['housing'].isin([1, 2, 3])).any()

Analiza wartości na kolumnie `housing` doprowadziła do powstania następującego wniosku: klienci, którzy wynajmowali mieszkanie mieli zazwyczaj większe problemy ze spłatą kredytu niż ci, którzy mieli mieszkanie własnościowe albo mieszkali za darmo.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='housing', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 16. Kolumna `number_credits`

Na kolumnie `number_credits` zawarte są informacje o liczbie zadłużeń kredytobiorców (wraz z tym kredytem) w tym banku. Znaczenie poszczególnych wartości:

* **1** - 1,
* **2** - 2-3,
* **3** - 4-5,
* **4** - więcej niż 5.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['number_credits'].isin([1, 2, 3, 4])).any()

Po analizie wartości na kolumnie `number_credits` można wywnioskować, że największe problemy występowały z klientami, którzy brali swój pierwszy albo drugi kredyt. W przypadku kolejnych kredytów problemy były raczej wyjątkową sytuacją, chociaż należy przy tym wniosku uwzględnić, że w zbiorze jest mało danych na temat takich klientów.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='number_credits', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 17. Kolumna `job`

Na kolumnie `job` zawarte są informacje o zatrudnieniu kredytobiorców. Znaczenie poszczególnych wartości:

* **1** - niezatrudniony / niewykwalifikowany - zamiejscowy,
* **2** - niewykwalifikowany - miejscowy,
* **3** - wykwalifikowany pracownik / urzędnik,                 
* **4** - menadżer / samozatrudniony / wysoko wykwalifikowany pracownik.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['job'].isin([1, 2, 3, 4])).any()

Po analizie danych na kolumnie `job` zauważono, że najczęściej problemy ze spłatą zadłużenia mieli pracownicy wykwalifikowani i urzędnicy. Sytuacja wygląda trochę lepiej w przypadku klientów niewykwalifikowanych oraz klientów, którzy byli samozatrudnieni, wysoko wykwalifikowani albo byli menadżerami.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='job', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 18. Kolumna `people_liable`

Na kolumnie `people_liable` zawarte są informacje o liczbie osób, które pozostają na utrzymaniu kredytobiorców. Znaczenie poszczególnych wartości:

* **1** - trzy lub więcej osób,
* **2** - od zera do dwóch osób.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['people_liable'].isin([1, 2])).any()

Po przeanalizowaniu danych na kolumnie `people_liable` okazało się, że klienci, którzy na swoim utrzymaniu mieli trzy lub więcej osób mieli mniejsze problemy ze spłatą zadłużenia niż klienci, którzy mieli od zera do dwóch osób na utrzymaniu.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='people_liable', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 19. Kolumna `telephone`

Na kolumnie `telephone` zawarte są informacje o tym, czy na kredytobiorcę jest zarejestrowany telefon stacjonarny. Znaczenie poszczególnych wartości:

* **1** - nie,                       
* **2** - tak.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['telephone'].isin([1, 2])).any()

Analiza wartości na kolumnie `telephone` doprowadziła do powstania następującego wniosku: klienci, którzy posiadali telefon stacjonarny mieli zazwyczaj większe problemy ze spłatą kredytu niż ci klienci, którzy nie mieli tego telefonu (wynika to prawdopodobnie z opłat z ten telefon).

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='people_liable', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 20. Kolumna `foreign_worker`

Na kolumnie `foreign_worker` zawarte są informacje o tym, czy kredytobiorca pracuje za granicą. Znaczenie poszczególnych wartości:

* **1** - tak,
* **2** - nie.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['foreign_worker'].isin([1, 2])).any()

Z analizy wartości na kolumnie `foreign_worker` wynika, że przypadków klientów, którzy pracowali za granicą i mieli problemy ze spłatą kredytu było bardzo mało, bo tylko 33 na 1000 przypadków. Zazwyczaj problemy ze spłatą kredytu mieli klienci, którzy nie pracowali za granicą.

In [ ]:
plt.figure(figsize=(7, 5))
plot = sns.countplot(data=df, x='foreign_worker', hue='credit_risk')
plt.ylabel('Liczba przypadków')
for rectangle in plot.patches:
    plot.text(rectangle.get_x() + rectangle.get_width() / 2, rectangle.get_height() + 0.75, rectangle.get_height(), horizontalalignment='center', fontsize=10)    

## 21. Kolumna `credit_risk`

Na kolumnie `credit_risk` zawarto informację o tym, czy kredytobiorca przestrzegał umowy kredytowej czy też nie. Znaczenie poszczególnych wartości:

* **0** - nie przestrzegał,
* **1** - przestrzegał.

Wartości na tej kolumnie nie wymagają czyszczenia, ponieważ wszystkie należą do zbioru wyżej opisanych wartości:

In [ ]:
np.logical_not(df['credit_risk'].isin([0, 1])).any()

Poniżej dostępny jest interaktywny wykres, gdzie można sprawdzić powiązanie pomiędzy wybraną kolumną, a ryzykiem kredytowym w sposób interaktywny.

In [ ]:
color_schemas = {
    'Plotly': px.colors.qualitative.Plotly,
    'Antique': px.colors.qualitative.Antique,
    'Bold': px.colors.qualitative.Bold,
    'Set1': px.colors.qualitative.Set1,
    'Vivid': px.colors.qualitative.Vivid,
}

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1('Związek wybranej kolumny z ryzykiem kredytowym'),
    daq.BooleanSwitch(
        id='barmode-switch',
        on=True,
        label='Grupowanie wartości',
        labelPosition='bottom'
    ),
    html.Label([
        'Kolumna:',
        dcc.Dropdown(
            id='column-dropdown',
            clearable=False,
            value='status',
            options=[
                {'label': x, 'value': x}
                for x in ['status', 'credit_history', 'purpose', 'savings', 'employment_duration', 'installment_rate', 'personal_status_sex', 'other_debtors', 'present_residence', 'property', 'other_installment_plans', 'housing', 'number_credits', 'job', 'people_liable', 'telephone', 'foreign_worker']
            ])
    ]),
    html.Label([
        'Schemat kolorów:',
        dcc.RadioItems(
            id='color-schema-radio',
            options=[
                {'label': x, 'value': x}
                for x in color_schemas.keys()
            ],
            value='Plotly'
        )
    ]),
    dcc.Graph(id='graph')
])

@app.callback(
    Output('graph', 'figure'),
    [
        Input('barmode-switch', 'on'),
        Input('column-dropdown', 'value'),
        Input('color-schema-radio', 'value')
    ]
)
def update_figure(barmode_switch, column, color_schema):
    histogram = px.histogram(
        df,
        x='credit_risk',
        color=column,
        barmode='group' if barmode_switch else 'relative',
        color_discrete_sequence=color_schemas[color_schema]
    )
    histogram.update_layout(
        xaxis_title='Ryzyko kredytowe',
        yaxis_title='Liczba przypadków',
        xaxis = dict(
            tickmode = 'array',
            tickvals = [0, 1],
            ticktext = ['Występuje', 'Nie występuje']
        )
    )
    
    return histogram

app.run_server(mode='inline')

# Zastosowanie algorytmów uczenia maszynowego do zadania klasyfikacji

Zastosowano trzy algorytmy uczenia maszynowego: naiwny klasyfikator Bayesa, drzewo decyzyjne oraz klasyfikator *k*-NN. Wszystkie trzy modele utworzone za pomocą wcześniej wymienionych algorytmów zostały wytrenowane na tych samych zbiorach treningowych i testowych (proporcje zbioru treningowego do testowego to 70:30).

In [ ]:
X = df.drop('credit_risk', axis=1)
y = df['credit_risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

## 1. Naiwny klasyfikator Bayesa

Model został stworzony z domyślnymi parametrami.

In [ ]:
gnb_model = GaussianNB()
gnb_model.fit(X, y)
gnb_y_predictions = gnb_model.predict(X_test)

print(classification_report(y_test, gnb_y_predictions))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, gnb_y_predictions)))

Otrzymany model, w postaci naiwnego klasyfikatora Bayesa, uzyskał dokładność na poziomie *0.76*.

## 2. Drzewo decyzyjne

Model został stworzony z następującymi parametrami:

* podział metodą entropii,
* maksymalna głębokość drzewa to 4.

In [ ]:
dtc_model = DecisionTreeClassifier(criterion="entropy", max_depth=4)
dtc_model.fit(X_train, y_train)
dtc_y_predictions = dtc_model.predict(X_test)

print(classification_report(y_test, dtc_y_predictions))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, dtc_y_predictions)))

Otrzymany model, w postaci drzewa decyzyjnego, uzyskał dokładność na poziomie *0.72*.

## 3. Klasyfikator *k*-NN

Model został stworzony z parametrem *n* o wartości 8.

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=8)
knn_model.fit(X_train, y_train)
knn_y_predictions = knn_model.predict(X_test)

print(classification_report(y_test, knn_y_predictions))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, knn_y_predictions)))

Otrzymany model, w postaci klasyfikatora *k*-NN, uzyskał dokładność na poziomie *0.68*.

# Podsumowanie

W niniejszym projekcie dokonano analizy zbioru, który zawiera dane na temat klientów banku i udzielanych im kredytów. Dane zawarte na poszczególnych kolumnach zostały opisane i przeanalizowane pod względem powiązań z ryzykiem kredytowym. Przeprowadzona analiza doprowadziła na powstania następujących wniosków:

1. Im więcej pieniędzy na koncie rozliczeniowym posiadał klient, tym większe prawdopodobieństwo, że wystąpią problemy ze spłatą zadłużenia.
2. Najpopularniejszym wyborem było zadłużenie się na czas do 12 miesięcy. Im dłuższy czas spłaty zobowiązania tym mniejsze zainteresowanie kredytobiorców.
3. Największe problemy ze spłatą kredytu mieli klienci, którzy wcześniej nie zaciągali kredytów.
4. Trzema najczęstszymi powodami zaciągnięcia kredytu były: kupno mebli/wyposażenia, kupno samochodu używanego oraz inne. Trzema najrzadszymi powodami zaciągnięcia kredytu były: wakacje, kupno radia/telewizora oraz biznes.
5. Najwięcej kredytów udzielono na kwoty do ok. 2500 DM, popularne były także kredyty powyżej ok. 2500 DM do ok. 5000 DM. Wyższe kwoty były zdecydowanie mniej popularne.
6. Największe problemy ze spłatą zadłużenia mieli klienci, którzy nie mieli żadnych oszczędności.
7. Największe problemy ze spłatą zadłużenia mieli klienci, którzy pracowali u ówczesnego pracodawcy od roku do 4 lat.
8. Największe problemy ze spłatą zadłużenia mieli klienci, u których miesięczna rata kredytu stanowiła mniej niż 20% pensji. U klientów, gdzie rata kredytu stanowiła więcej niż 20% pensji występowały mniejsze problemy ze spłatą zadłużenia.
9. Największe problemy ze spłatą zadłużenia mieli mężczyźni, którzy byli żonaci albo byli wdowcami.
10. Można zauważyć, że większość kredytów była związana z tylko jedną osobą (bez współwnioskodawców czy żyrantów).
11. Im dłużej klient mieszkał w swoim miejscu zamieszkania, tym większe było prawdopodobieństwo problemów ze spłatą zadłużenia.
12. Klienci, którzy posiadali nieruchomość mieli zazwyczaj mniejsze problemy ze spłatą zadłużenia.
13. Najczęściej kredytobiorcami były osoby w wieku 25-35 lat. Popularność kredytów wśród osób starszych niż 35 lat spada, aż do najmniejszej wartości dla osób w wieku powyżej 70 lat.
14. Okazuje się, że posiadanie innych zobowiązań kredytowych często nie było równoznaczne z problemami ze spłatą kredytu przez klientów.
15. Klienci, którzy wynajmowali mieszkanie mieli zazwyczaj większe problemy ze spłatą kredytu niż ci, którzy mieli mieszkanie własnościowe albo mieszkali za darmo.
16. Największe problemy występowały z klientami, którzy brali swój pierwszy albo drugi kredyt.
17. Najczęściej problemy ze spłatą zadłużenia mieli pracownicy wykwalifikowani i urzędnicy.
18. Klienci, którzy na swoim utrzymaniu mieli trzy lub więcej osób mieli mniejsze problemy ze spłatą zadłużenia niż klienci, którzy mieli od zera do dwóch osób na utrzymaniu.
19. Klienci, którzy posiadali telefon stacjonarny mieli zazwyczaj większe problemy ze spłatą kredytu niż ci klienci, którzy nie mieli tego telefonu (wynika to prawdopodobnie z opłat z ten telefon).
20. Przypadków klientów, którzy pracowali za granicą i mieli problemy ze spłatą kredytu było bardzo mało. Problemy występowały głównie z klientami nie pracującymi za granicą.

Stworzono także trzy klasyfikatory za pomocą algorytmów: naiwny klasyfikator Bayesa, drzewo decyzyjne oraz klasyfikator *k*-NN. Średnia jakość klasyfikacji otrzymanych klasyfikatorów wyniosła *0.72*.